# Práctica 1: Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación\*

### Minería de Datos: Curso académico 2020-2021

### Profesorado:

* Juan Carlos Alfaro Jiménez
* José Antonio Gámez Martín

\* Adaptado de las prácticas de Jacinto Arias Martínez y Enrique González Rodrigo


### Grupo H:

* Alejandro Fernández Arjona
* Pablo Torrijos Arenas

En esta práctica hemos trabajado algunos de los aspectos más importantes del proceso *KDD* (*Knowledge Discovery from Data*):

* Almacenamiento y carga de datos
* Análisis exploratorio de datos
* Preprocesamiento de datos
* Validación de modelos de clasificación

Para la visualización de los datos, además de las librerías `pandas` y `plotly`, hemos usado algunas librerías auxiliares, como `seaborn` y `graphviz`. Para los algoritmos de clasificación hemos usado `scikit-learn`.

Hemos realizado los estudios sobre las base de datos `pima_diabetes` y `wisconsin`:

- `pima_diabetes`: https://www.kaggle.com/uciml/pima-indians-diabetes-database
- `wisconsin`: https://www.kaggle.com/uciml/breast-cancer-wisconsin-data

---

Vamos a empezar realizando nuestro análisis para la base de datos `pima_diabetes`.

# Pima Indians Diabetes Database

## 1. Preliminares

Cargamos las librerías que vamos a usar posteriormente:

In [ ]:
# Third party
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.tree import DecisionTreeClassifier

#Otras liberrías auxiliares
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from sklearn import tree

from imblearn import FunctionSampler
from imblearn.pipeline import make_pipeline

import numpy as np
import pandas as pd

import seaborn as sb
import plotly as plt
import plotly.express as px
import plotly.graph_objects as go
import graphviz 
import matplotlib
import matplotlib.pyplot as pypl
from matplotlib.pyplot import figure

# Local application
import miner_a_de_datos_an_lisis_exploratorio_utilidad as utils

Además, fijamos una semilla para que los experimentos sean reproducibles (hemos usado la misma que se usó en la libreta del estudio de `iris`:

In [ ]:
seed = 27912

## 2. Acceso y almacenamiento de datos

El primer conjunto de datos que usaremos es `diabetes`. Es original del Instituto Nacional de Diabetes y Enfermedades Digestivas y del riñón. El objetivo es intentar predecir si un paciente tiene o no diabetes, basándonos en ciertas variables predictores. Se trata de 768 instancias, mujeres de entre 20 y 81 años, un extracto muy pequeño de la colección de datos original.

La variable objetivo es `Outcome`, y estos son los posibles Outcomes:

* `0`: Predicción de que el paciente no tiene diabetes.
* `1`: Predicción de que el paciente tiene diabetes.

Las distintas variables predictoras son las siguientes:

* `Pregnancies`: Número de embarazos.
* `Glucose`: Concentración de glucosa en plasma (tras 2 horas de un test de tolerancia a glucosa oral).
* `BloodPressure`: Presión arterial diastólica (en milímetros de mercurio).
* `SkinThickness`: Espesor del pliegue cutáneo del tríceps (en milímetros).
* `Insulin`: Insulina en suero 2-horas.
* `BMI`: Índice de masa corporal (kg / m^2).
* `DiabetesPedigree`: Función de pedigrí de diabetes.
* `Age`: Edad (en años).

El objetivo sería clasificar una nueva instancia (cuyo `Outcome` es desconocido) en función de sus propiedades.

Comenzamos cargando el conjunto de datos `diabetes`:

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv"

index = None
target = "Outcome"

data = utils.load_data(filepath, index, target)


Tenemos que darnos cuenta de que el conjunto de datos `diabetes` no tiene una variable identificadora, por lo que definimos como identificador `None` y el método creará una variable Id automáticamente. Definimos como variable objetivo `Outcome`.

Una vez hemos cargado el conjunto de datos es fundamental comprobar que el proceso ha funcionado sin problemas, y que las variables y los valores están dentro de lo esperado. Para ello, podemos escoger una instancia al azar o mostrar las primeras instancias del conjunto de datos.

Podemos usar la función `sample` para obtener una muestra aleatoria de `n` instancias del conjunto de datos, ya que con el método `head` obtendríamos una muestra muy sesgada:

In [ ]:
data.sample(5, random_state=seed)

Es muy útil disponer del conjunto de datos separado dos subconjuntos, uno con las variables predictoras (`X`) y otro con la variable objetivo (`y`). Se puede utilizar el siguiente fragmento de código para dividirlo: 

In [ ]:
(X, y) = utils.divide_dataset(data, target="Outcome")

De nuevo, comprobamos que se haya separado correctamente. Comenzamos con las variables predictoras:

In [ ]:
X.sample(5, random_state=seed)

Y continuamos con la variable clase:

In [ ]:
y.sample(5, random_state=seed)

Si bien podríamos comenzar con el análisis exploratorio, vamos a dividir primero nuestro conjunto de datos en dos:

* Una muestra de entrenamiento (vamos a usar 75%)
* Una muestra de prueba (el 25% restante)

De este modo, podemos dejar el conjunto de prueba a modo de instancias no observadas para asegurarnos que los resultados de validación han sido estimados de manera honesta (y no optimista). De hecho, si utilizamos el mismo conjunto de datos para aprender y validar un modelo, observaremos un resultado inusual y es que, conforme más sobreajustado está el modelo, menor es el error cometido. Por el contrario, si usamos un conjunto de entrenamiento muy pequeño (50% training, 50% test, por ejemplo), no estaríamos ajustando lo suficiente (*underfitting*)y podemos obtener resultados peores de los que deberíamos.

Para realizar un *holdout* podemos utilizar el método `train_test_split` de `scikit-learn`:

Podríamos realizar una validación cruzada con k=5 por ejemplo (carpetas de unas 154 o 153 instancias) para reducir la aleatoriedad y usar cada registro como test una vez, pero lo dejamos para la siguiente práctica, para esta realizaremos un *holdout*.

In [ ]:
train_size = 0.75

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

Aleatorizamos las instancias del conjunto de datos (`shuffle=True`, valor por defecto aunque no lo especifiquemos) para evitar que eliminar todas las instancias de alguna clase en conjuntos de datos que estén ordenados por la variable clase. No es nuestro caso, ya que la variable `Outcome` toma valores 0 y 1 alternadamente a lo largo de los 768 registros, pero no está mal que aleatoricemos los datos igualmente.

Mediante la semilla, con `random_state`, conseguimos las mismas particiones de training y test cada vez que ejecutemos el algoritmo, para conseguir la reproducibilidad de los experimentos.

Al igual que en el caso que se nos dio de `iris`, hemos aplicado un *holdout* estratificado (`stratify=y`), para conservar la proporción de ejemplos de cada clase durante la revisión. Es muy importante que lo hagamos con esta base de datos, ya que al ser un problema desbalanceado, podríamos eliminar mucha información si no lo hacemos.

De nuevo, vamos a asegurarnos de que el conjunto de datos se ha dividido correctamente en entrenamiento y prueba. Comenzamos con las variables predictoras del conjunto de datos de entrenamiento:

In [ ]:
X_train.sample(5, random_state=seed)

Y prueba:

In [ ]:
X_test.sample(5, random_state=seed)

Por último, finalizamos con la variable objetivo del conjunto de datos de entrenamiento:

In [ ]:
y_train.sample(5, random_state=seed)

Y prueba:

In [ ]:
y_test.sample(5, random_state=seed)

Para facilitar el análisis exploratorio de datos, juntamos de nuevo las variables predictores con la variable clase.

In [ ]:
data_train = utils.join_dataset(X_train, y_train)

data_test = utils.join_dataset(X_test, y_test)

Vamos a obtener una muestra aleatoria de ambos conjuntos:

In [ ]:
data_train.sample(5, random_state=seed)

In [ ]:
data_test.sample(5, random_state=seed)

## 3. Análisis exploratorio de datos

Antes de comenzar el preprocesamiento, vamos a analizar las variables y sus relaciones, mediante gráficos y estadísticos. Usamos las libreías `Pandas`, `Plotly` y `Seaborn`.

## 3.1 Descripción del conjunto de datos

Como hemos dicho antes, el conjunto de datos `Diabetes` tiene 768 instancias y 9 variables. Vamos a ver esta información sobre el conjunto de entrenamiento con el atributo shape.

In [ ]:
data_train.shape

Tal y como se puede observar, el conjunto de datos de entrenamiento está formado por 576 casos y 9 variables (8 variables predictoras y 1 variable clase, Outcome).

Para conocer el tipo de las variables usamos `info`:

In [ ]:
data_train.info(memory_usage=False)


Es decir, 6 variables predictoras del conjunto de datos (`Pregnancies`,`Glucose`,`BloodPressure`,`SkinThickness`,`Insulin`,`Age`) son numéricas (continuas) del tipo `int64`, 2 variables predictoras numéricas (continuas) del tipo `float64`, y la variable clase (`Outcome`) es categórica (`category`) o discreta, con los estados `0` y `1`, como vemos a continuación:

In [ ]:
y.cat.categories

Es decir, nuestra variable clase es bivariada (tiene 2 posibles valores).

## 3.2 Visualización de las variables

Hemos realizado distintos tipos de gráficas y de diagramas para comprender mejor las variables de nuestro problema y las capadidades de la librería `plotly`, realizando tanto un análisis univariado, como uno multivariado.

Análisis univariado (involucra una sola variable):
* Histogramas para las variables numéricas
* Diagramas circulares para observar información de `Insulin` y `SkinThickness`
* Diagramas de barras para la variable clase `Outcome`
* Diagramas de cajas
* Diagramas de violín


Análisis multivariado (involucra varias variables):
* Matriz de gráficos de nube de puntos
* Mapa de calor
* Gráficos de dispersión
* Diagramas de  densidad de contorno

### 3.2.1 Análisis univariado

Para empezar, vamos a hacer un análisis univariado para identificar ruido y outliers en las variables de nuestro conjunto de datos.

In [ ]:
utils.plot_histogram(data_train)

La variable `Pregnancies` sigue una distribución normal con asimetría positiva, y la mayoría de casos se encuentran entre 0 y 10 embarazos. No parece muy lógico que haya tantos casos de mujeres con más de 4 casos, ya que en mi opinión la gran mayoría de mujeres suelen tener entre 0 y 4 embarazos, no es lo normal que de 768 mujeres 38 de ellas hayan pasado por 8 embarazos. Esos datos los tendremos en cuenta aunque no parezcan lógicos, ya que tampoco podemos considerarlos como outlayers ni como ruido. Lo que sí podemos considerar como outlayers son las mujeres que hayan tenido más de 12 embarazos, llegando incluso a 17. Estos valores los podemos eliminar.

Las variables `Glucose`, `BloodPressure`, `BMI` y `DiabetesPedigreeFunction` muestran una distribución con tendencia central (teniendo la última de ellas una asimetría positiva), es decir, también siguen una distribución normal. También debemos destacar que hay algunos casos cuyo nivel de glucosa, índice de masa corporal o presión sanguínea es 0, lo cual significa que se trata de valores perdidos y debemos imputarlos.  En cuanto a `BMI` y `DiabetesPedigreeFunction` también podemos destacar que hay algunos outlayers que debemos eliminar. Por ejemplo, en cuanto al índice de masa corporal, aunque lo más común son los valores entre 20 y 40, existen casos con mayor IMC, pero no tanto como 60 o 68 como se ve en la gráfica, ya que la obesidad extrema comprende desde 40 hasta 55 de IMC. En cuanto a la función de pedigree de diabetes, también hay unos cuantos valores outlayers que eliminaremos más adelante. 

Como se puede observar, la variable `Insulina` parece seguir una distribución exponencial, tieniendo la mayoría de los casos valor `0`. Sin embargo, si nos paramos a analizar un poco el significado de esta gráfica, nos damos cuenta de que todos esos datos con valor de insulina igual a cero son erróneos. Una persona no puede generar cero de insulina, moriría, por lo que se trata de valores perdidos. En las variables que he comentado antes vamos a tratar de imputar los valores perdidos, sin embargo, en este caso, esos errores suponen el 49.8% de la variable, por lo que estaríamos tratando con unos datos muy sesgados. En este caso, vamos a eliminar la variable y a no tenerla en cuenta para nuestro problema.

Por último, la variable `SkinThickness` (distribución normal), que mide el grosor de la piel, también tiene una gran cantidad de datos perdidos: suponen el 32.3% del total. Además, no parece normal que el grosor de la piel esté concentrado entre 15 y 40 milímetros, ya que lo normal es entre 0.5mm y 4.0mm. En cualquier caso, esa enorme cantidad de datos perdidos es motivo suficiente para eliminar la variable y no tenerla en cuenta, al igual que hemos comentado respecto a la variable `Insulina`.

Por último, la variable `Age` presenta de nuevo una distribución normal, y en este caso no parece haber valores perdidos ni outlayers, ya que todas las edades se encuentran entre 20 y 80 años.

Aunque hemos calculado el número de valores perdidos manualmente, vamos a ver ahora esa cantidad con un gráfico circular. Empezamos con `Insuline`:

In [ ]:
circular1 = px.pie(data_train, values='Outcome', names='Insulin')
circular1.show()

Aunque los registros con valor mayor que cero no se aprecien bien, se ve claramente que casi la mitad de las instancias toman valor cero, por lo que, como ya hemos dicho, no debemos tener en cuenta esta variable predictora.

Repetimos para `SkinThickness`:

In [ ]:
circular1 = px.pie(data_train, values='Outcome', names='SkinThickness')
circular1.show()

En este caso, casi un tercio de los datos son datos erróneos, por lo que también tenemos que eliminar esta variable.

Vamos a almacenar en una lista las dos variables a eliminar, para futuros usos.

In [ ]:
eliminadas = ['Insulin','SkinThickness']

Continuamos visualizando la variables clase:

In [ ]:
utils.plot_barplot(data_train) 

Podemos observar que la clase 0 (No diabetes) tiene el 65.1% de los casos, y la clase 1 (Sí diabetes) el resto de los casos, es decir, 34.9%. Esto quiere decir que el problema no está `balanceado`.


Podemos realizar otro diagrama de barras con el conjunto de datos original (data) en lugar de usar el conjunto de datos de entrenamiento (data_train), para comprobar si habíamos estratificado correctamente:

In [ ]:
utils.plot_barplot(data)

Como se puede observar, se matiene el mismo porcentaje de ceros y unos en ambos casos, por lo que la estratificación es correcta.

Vamos a realizar un diagrama de cajas para observar las variables predictoras y sus cuartiles. Los diagramas de cajas sirven también para encontrar los outliers rápidamente. Usamos el conjunto X_train ya que la variable clase no nos interesa analizarla en este caso, siempre toma valores 0 o 1.

In [ ]:
cajas2 = go.Figure()

for columna in X_train:
    cajas2.add_trace(go.Box(y=X_train[columna].values, name=X_train[columna].name))

cajas2.show()

A simple vista en este diagrama podemos ver cómo por culpa de todos los valores perdidos de la variable `Insulina`, ésta tiene su mediana en el valor 0, siendo los valores superiores a 326 los outliers, cuando no es para nada lo que ocurre en realidad. Como vamos a eliminar las variables `Insulina` y `SkinThickness`, podemos realizar otro diagrama de cajas con el resto de variables.

In [ ]:
cajas2 = go.Figure()

for columna in X_train:
    if columna not in eliminadas:
        cajas2.add_trace(go.Box(y=X_train[columna].values, name=X_train[columna].name))

cajas2.show()

Podemos destacar que `Glucose`, `BloodPressure` y `BMI` son variables simétricas, ya que la mediana se encuentra en el centro del rectángulo. Respecto a esas 3 variables, también podemos comprobar lo que dijimos antes, que tienen algunos datos perdidos que toman valor 0. `Pregnancies` y `DiabetesPedigreeFunction` también tienen registros con valor 0, pero como ya hemos dicho antes, en estas variables es completamente normal.

En cuanto a `Pregnancies`, salen valores muy altos, ya que por ejemplo según este diagrama la mediana se encuentra en 3, y existen múltiples registros con valor por encima de 10 embarazos.

Por último, se pueden ver outlayers en todas las variables, excepto en la de `Glucose`

Vamos a realizar ahora un diagrama de violín para `Glucose` por ejemplo. Es similar a un diagrama de cajas pero añadiendo densidad a ambos lados del diagrama:

In [ ]:
fig = px.violin(data_train, y="Glucose")
fig.show()

Podemos observar, al igual que en el diagrama de cajas, que la mayoría de los registros se encuentran en el intervalo [100-150]. Además, vemos como hay esos registros con valor 0 que deberemos imputar. 

### 3.2.2 Análisis multivariado

Hemos realizado ahora un análisis multivariado para tratar de obtener mejores conclusiones determinando la potencia discriminativa de los atributos, viendo las relaciones entre ellos respecto a la información que nos den sobre la variable objetivo `Outcome`.

Para empezar, vamos a crear una matriz de gráficos del tipo nube puntos, al igual que se hizo en el estudio sobre `Iris`. Cada diagrama muestra la relación entre pares de variables predictoras, coloreando cada registro según la clase a la que pertenezca (0 o 1).

In [ ]:
utils.plot_pairplot(data_train, target="Outcome")

Este gráfico resultaba muy útil en problemas como el de base de datos `iris`, ya que al tener pocas variables y pocos registros se podían extraer rápidamente algunas conclusiones. Sin embargo, en este problema es difícil sacar conclusiones viendo este pairplot debido a que hay 8 x 8 (64) gráficas y no se aprecian muy bien. Podemos ver algunos outliers en casi todos los diagramas.

Vamos a realizar ahora un mapa de calor (heatmap) para ver mejor la correlación que existe entre las distintas variables predictoras de nuestra base de datos:

In [ ]:
import plotly.express as px
px.imshow(data_train.corr(method="pearson"))

Como se puede apreciar, no existe prácticamente correlación entre ningun par de variables de nuestro problema. La única pareja que supera el umbral 0.5 es la correlación entre edad y número de embarazos. Tiene sentido que a mayor edad, más número de embarazos se tengan, pero realmente un valor de correlación de 0.558 no es una correlación muy fuerte.

Podemos crear ahora un gráfico de dispersión entre `Age` y `Pregnancies`, para comprobar la relación que hemos comentado que existía entre esas 2 variables. Separamos con colores según el `Outcome` sea 0 o 1.

In [ ]:
dispersion = px.scatter(data_train, x="Age",y="Pregnancies",color="Outcome",trendline="ols")
dispersion.show()

Como habíamos dicho, existía una correlación pequeña, y aquí podemos apreciar que hay mucha dispersión entre las variables. Esto se debe a que las mujeres no tienen una edad fija a la que quedarse embarazadas, ni un número fijo de embarazos, sino que cada caso es muy diferente, aunque exista cierta correlación.

Podemos realizar un diagrama de densidad de contorno (o histograma en 2D) para comparar de otra manera estas 2 variables. Este gráfico se utiliza cuando hay muchos puntos en un diagrama de dispersión, como el que acabamos de hacer.

In [ ]:
contorno = px.density_contour(data_train, x="Age", y="Pregnancies")
contorno.show()

Podemos crear ahora otro gráfico de dispersión y otro de densidad de contorno entre dos variables que tengan correlación casi nula (0), para ver la diferencia respecto a estos dos diagramas. Por ejemplo, `Pregnancies` y `DiabetesPedigreeFunction`.


In [ ]:
dispersion2 = px.scatter(data_train, x="DiabetesPedigreeFunction",y="Pregnancies",color="Outcome",trendline="ols")
dispersion2.show()

In [ ]:
contorno2 = px.density_contour(data_train, x="DiabetesPedigreeFunction", y="Pregnancies")
contorno2.show()

Efectivamente, en este caso existe una dispersión todavía mayor, debido a que es completamente indeferente el número de embarazos que una mujer haya tenido, con su función de pedigrí de diabetes.

## 4. Preprocesamiento de datos

Ahora comenzamos con la tarea más importante de toda la práctica, el preprocesamiento de los datos. Vamos a transdormar los datos crudos en información más accesible para los algoritmos de aprendizaje. Vamos a realizar una limpieza y una discretización del conjunto de datos, todo ello dentro de un pipeline, para evitar fugas de datos.

## 4.1 Limpieza de datos

Para la limpieza de datos en nuestro problema vamos a realizar los siguientes pasos:
* Eliminar las variables `Insuline` y `SkinThickness` por tener más de un 20% de datos perdidos.
* Imputar datos perdidos de las variables `Glucose`, `BloodPressure` y `BMI`.
* Eliminar *outliers*

### Eliminar variables

Para comenzar con la limpieza de los datos, lo primero que vamos a hacer es eliminar las variables `Insuline` y `SkinThickness`, como hemos explicado en el análisis exploratorio. Podemos eliminar directamente las variables con la función drop de pandas:

In [ ]:
eliminadas = ['Insulin','SkinThickness']

copiadatos = X_train.copy()
copiadatos = copiadatos.drop(eliminadas, axis=1)
copiadatos.sample(5)

Sin embargo, queremos eliminar las variables dentro del pipeline, al igual que la normalización. Para ello, tenemos que crear una clase que implemente los métodos fit( ) y transform( ), y el constructor debe recibir como parámetro las columnas a eliminar.

In [ ]:
class EliminarVariables():
    
    def __init__(self, columnas):
        self.columnas=columnas
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, x, y=None):
        return x.drop(self.columnas,axis=1,inplace=False)
        

### Imputar datos

Queremos imputar los datos de las variables `Glucose`, `BloodPressure` y `BMI`, como ya hemos dicho antes. Para ello necesitamos usar ColumnTransformer, al cual debemos indicarle qué tipo de imputador queremos usar (por ejemplo, SimpleImputer con la media), y las variables que queremos modificar.

In [ ]:
variables = ['Glucose','BloodPressure','BMI']

Imputador = ColumnTransformer([('imp1',SimpleImputer(missing_values=0,strategy="mean"), variables)])

### Eliminar outliers

Para eliminar outliers vamos a usar la función que se nos proporcionó en prácticas. Creamos un IsolationForest para detectar los outliers, y con un FunctionSampler la incluiremos en el pipeline. Es importante que el valor `random_state` lo igualemos a nuestra semilla, para permitir que los experimentos se puedan reproducir.

In [ ]:
def outlier_rejection(X, y):
    model = IsolationForest(max_samples=100,
                            contamination=0.4,
                            random_state=seed)
    model.fit(X)
    y_pred = model.predict(X)
    return X[y_pred == 1], y[y_pred == 1]


EliminarOutliers = FunctionSampler(func=outlier_rejection)

## 4.2 Discretización

En el ejemplo de `iris` era evidente que dividir en 3 secciones era una buena opción viendo las gráficas, por que las 3 clases estaban muy diferenciadas, pero en el caso de `Diabetes`, no se puede extraer una información clara de las gráficas que hemos visto, por lo que usamos el mismo discretizador que usábamos en `iris`: discretización uniforme, dividiendo en 3 intervalos de igual anchura.

In [ ]:
Discretizador = KBinsDiscretizer(n_bins=3, strategy="uniform")

## 5. Algoritmos de clasificación y evaluación de modelos

## 5.1 Algoritmos de clasificación

### Algoritmo *Zero-R*

Vamos a usar primero el algoritmo Zero-R, aunque no tenga mucha precisión, nos puede dar una idea para la precisión que debemos conseguir con otros modelos. Para usar el algoritmo Zero-R, recurrimos al estimador `DummyClassifier` de `scikit-learn`:

In [ ]:
ZeroR = DummyClassifier(strategy="most_frequent")

### Inducción de árboles de decisión

Vamos a usar ahora un árbol de decisión, usamos el estimador `DecisionTreeClassifier` de `scikit-learn`. Usamos la misma semilla, como siempre, para que las pruebas sean reprodubibles.

In [ ]:
ArbolDecision = DecisionTreeClassifier(random_state=seed)

### *Pipeline*

Para crear un *pipeline*, vamos a usar la función `make_pipeline` de `scikit-learn`. Esta toma como parámetros la lista de transformadores a aplicar al conjunto de datos y, al final de este, el estimador a utilizar.

Los transformadores de limpieza que hemos son:
* Eliminar las variables `Insulin` y `SkinThickness`.
* Imputar valores perdidos de `Glucose`, `BloodPressure` y `BMI`.
* Eliminar outliers de todas las variables que los tengan.

Vamos a evaluar 5 modelos distintos para comprobar como afectan los distintas transformadores a los resultados:
* Usando el algoritmo `ZeroR`.
* Usando el algoritmo `DecisionTreeClassifier`.
* Pipeline usando `DecisionTreeClassifier` y los transformadores de limpieza.
* Pipeline usando `DecisionTreeClassifier` y `KBinsDiscretizer`.
* Pipeline usando `DecisionTreeClassifier`, los transformadores de limpieza y `KBinsDiscretizer`.

In [ ]:
eliminadas = ['Insulin','SkinThickness']

Pipeline1 = make_pipeline(EliminarVariables(eliminadas),Imputador,EliminarOutliers, ArbolDecision)
Pipeline2 = make_pipeline(Discretizador, ArbolDecision)
Pipeline3 = make_pipeline(EliminarVariables(eliminadas),Imputador,EliminarOutliers, Discretizador, ArbolDecision)

## 5.2 Evaluación de modelos

Ahora es el momento de entrenar y validar nuestros clasificadores. Para ello, vamos a usar una matriz de confusión y tasa de acierto.

### Zero_R

In [ ]:
utils.evaluate(ZeroR,
               X_train, X_test,
               y_train, y_test)

Como era de esperar, el modelo *Zero-R* obtiene malos resultados, solo predice la clase mayoritaria (0). Si la clase estuviera balanceada, obtendríamos una precisión todavía peor, del 50%. Vamos a probar otros modelos para ver cuánto mejora la precisión.

### Árbol de decisión

In [ ]:
utils.evaluate(ArbolDecision,
               X_train, X_test,
               y_train, y_test)

### Árbol de decisión con los transformadores de limpieza

In [ ]:
utils.evaluate(Pipeline1,
               X_train, X_test,
               y_train, y_test)

### Árbol de decisión discretizando el conjunto de datos

In [ ]:
utils.evaluate(Pipeline2,
               X_train, X_test,
               y_train, y_test)

### Árbol de decisión discretizando el conjunto de datos y usando los transformadores de limpieza

In [ ]:
utils.evaluate(Pipeline3,
               X_train, X_test,
               y_train, y_test)

Como era evidente, los árboles de decisión obtienen mejores resultados que Zero_R, que tiene un 65.1% de precisión, lo cual es lógico teniendo en cuenta que ese es el porcentaje de ceros que hay en la variable clase; tanto en el conjunto de datos original, como en el conjunto de entrenamiento (gracias a que hemos estratificado correctamente). No importa que discreticemos o no, ni que usemos o no los transformadores de limpieza, todos los modelos con árbol de clasificación superan al algoritmo Zero_R.

Usando simplemente árboles de decisión, obtenemos una tasa de acierto del 66.67%, lo cual supone una mejora de más de 1.5% respecto al Zero_R.

Si usamos también los transformadores de limpieza, obtenemos una precisión del 67.2%, que indica una mejora del 0.5% aproximadamente respecto a no usar esos transformadores.

Si usamos árboles de decisión y discretizamos el conjunto de datos (al igual que se hacía en la libreta de `iris`), se obtiene una tasa de acierto de 68.22%, mejorando en un 1.5% a su versión sin discretizar.

Y por último, si con árboles de decisión usamos los transformadores de limpieza y discretizamos el conjunto de datos, conseguimos la mayor precisión de todas, 71.35%.

Estos resultados pueden variar mucho según la semilla que elijamos (por la aleatoriedad) y porque estas base de datos con las que estamos trabajando son muy pequeñas, y fallar o acertar la predicción en un par de registros puede variar mucho los resultados. Para obtener resultados más fiables, podríamos ejecutar estos mismos algoritmos con 100 semillas distintas (por ejemplo), y devolver como medida de precisión la media de todas esas pruebas. Otra opción sería realizar una validación cruzada, para evitar resultados demasiado buenos o demasiado malos.

---

Ahora vamos a realizar los mismos pasos, pero para la base de datos `Wisconsin`.

# Breast Cancer Wisconsin (Diagnostic) Data Set

## 1. Preliminares

Vamos a usar los mismos *imports* y la misma semilla que en el análisis exploratorio de la base de datos `pima_diabetes`, por lo que no debemos realizar nada en este apartado.

## 2. Acceso y almacenamiento de datos

Ahora, vamos a cargar los datos de la base de datos desde el fichero .csv, y definimos el nombre de la columna de id y de la clase.

In [ ]:
filepath = "../input/breast-cancer-wisconsin-data/data.csv"

index = "id"
target = "diagnosis"


data = utils.load_data(filepath, index, target)

Comprobamos que la carga se ha realizado correctamente, mediante un muestreo de 5 instancias:

In [ ]:
data.sample(5, random_state=seed)

En el muestreo anterior podemos ver cómo hay una variable cuyo nombre es `Unnamed 32`. Lo volvemos a comprobar con data.info:

In [ ]:
data.info(memory_usage=False)

Y podemos ver cómo se crea una última columna sin ningún dato ni nombre. Esto es debido a que en el fichero .csv la línea con el nombre de las variables acaba con una coma, por lo que Pandas detecta una variable más sin ningún nombre especificado. Para arreglarlo, simplemente borramos esa columna, aunque también podríamos haber modificado el archivo .csv.

In [ ]:
data = data.drop(data.columns[[31]], axis='columns')
data.info(memory_usage=False)

Como podemos ver, ahora ya contamos con las 31 columnas que tenemos que tener.

Ahora, vamos a dividir nuestra base de datos en los conjuntos de variables predictoras (`X`), y variables predictivas (`y`).

In [ ]:
(X, y) = utils.divide_dataset(data, target="diagnosis")

Al igual que antes, comprobamos que el conjunto de datos se haya separado correctamente:

In [ ]:
X.sample(5, random_state=seed)

In [ ]:
y.sample(5, random_state=seed)

Y ahora dividimos nuestros datos en los conjuntos de entrenamiento y prueba para `X` e `y`, con unos porcentajes de 70% entrenamiento y 30% prueba. Además, al hacerlo los datos se aleatorizan, evitando así problemas con bases de datos ordenadas.

In [ ]:
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

Y comprobamos que los 4 conjuntos resultantes son correctos:

In [ ]:
X_train.sample(5, random_state=seed)

In [ ]:
y_train.sample(5, random_state=seed)

In [ ]:
X_test.sample(5, random_state=seed)

In [ ]:
y_test.sample(5, random_state=seed)

Como parecen correctos, para facilitar el análisis exploratorio posterior, vamos a juntar tanto X_train e y_train, como X_test e y_test:

In [ ]:
data_train = utils.join_dataset(X_train, y_train)
data_test = utils.join_dataset(X_test, y_test)

Y volvemos a comprobar que los conjuntos creados son correctos:

In [ ]:
data_train.sample(5, random_state=seed)

In [ ]:
data_test.sample(5, random_state=seed)

Y al igual que antes, también parece que se han realizado las uniones correctamente.

---

## 3. Análisis exploratorio de datos

Primero vamos a ver el tamaño de nuestro problema, conociendo el tamaño los conjuntos de datos que hemos creado, y los tipos de variables que éstos tienen.

### 3.1 Descripción del conjunto de datos

Primero vamos a ver el tamaño de nuestro problema, conociendo el tamaño los conjuntos de datos que hemos creado, y los tipos de variables que éstos tienen.

In [ ]:
data_train.info(memory_usage=False)

In [ ]:
utils.plot_barplot(data_train)

Podemos ver que tenemos 398 instancias, en las cuales contamos con 30 variables predictoras numéricas, y una variable clase categórica (la variable clase). Además, esta variable clase tiene 2 estados posibles, `B` y `M`, habiendo bastantes más casos de `B` que de `M` por lo que nuestra muestra no está balanceada.

En cuanto a las variables predictoras, se dividen en 3 conjuntos atendiendo a la nomenclatura de las variables:
* Medias: Las 10 primeras variables son medias de distintos parámetros de las células.
* Desviación típica: Las 10 siguientes son las desviaciones típicas de dichos parámetros.
* Peor: Y las 10 últimas son el peor de los casos para cada variable, de entre las células observadas.

### 3.2 Visualización de las variables

---

### 3.2.1 Análisis multivariado

Primero vamos a comprobar la correlación de las variables predictoras realizando un análisis multivariado, ya que como tenemos una gran cantidad de variables (30), si podemos deberíamos intentar reducir ese número antes de iniciar una análisis univariado.

Podríamos usar esta función de `plotly` para hacerlo, pero se ve más claro con `Seaborn`.
> px.imshow(data_train.corr(method="pearson"))

In [ ]:
figure(figsize=(20,15))

sb.heatmap(data_train.corr(), annot=True)

De este análisis de correlación podemos obtener conclusiones valiosas (poniendo el umbral entre muy correlacionadas y no en 0,9):
* Por un lado, las variables `radius_mean`, `perimeter_mean` y `area_mean` están muy correlacionadas, lo cual tiene mucho sentido viendo que tanto el perímetro como el área son funciones matemáticas basadas en la multiplicación del radio por un número. Lo mismo pasa entre `radius_se`, `perimeter_se` y `area_se`; y entre `radius_worst`, `perimeter_worst` y `area_worst`. Por lógica, nos vamos a quedar con los valores del radio ya que los otros están basados en él. Por tanto, nos quedamos solo con `radius_mean`, `radius_sd` y `radius_worst`.
* Además, la media del radio `radius_mean` está muy correlacionada con su peor valor `radius_worst`, por lo que podemos eliminar este último.
* Con `texture_mean` y `texture_worst` pasa exactamente lo mismo, por lo que también eliminaremos `texture_worst`.
* `concavity_mean`, `concave points_mean`, `compactness_mean` están muy relacionadas, por lo que nos quedamos con `concavity_mean`, y con `concavity_worst`, `concave points_worst` y `compactness_worst` ocurre lo mismo, quedándonos con `concavity_worst`.
* Pero además, `concavity_mean` y `concavity_worst` también están muy correlacionadas, por lo que eliminamos esta última.

Por ello, vamos a eliminar las variables comentadas anteriormente de `data_train`, simplificando así la base de datos perdiendo la mínima información posible. 

Esto lo hacemos para continuar la visualización de las variables solo con las que nos interesan, aunque posteriormente hagamos el borrado dentro del *pipeline* ya que no usaremos `data_train` si no `X_train` e `y_train`.

In [ ]:
borrado = ['perimeter_mean', 'area_mean', 'perimeter_se', 'area_se', 'perimeter_worst', 'area_worst', 
           'area_worst', 'radius_worst', 'texture_worst', 'concave points_mean', 'compactness_mean', 
           'concave points_worst', 'compactness_worst', 'concavity_worst']

data_train.drop(borrado, axis='columns', inplace=True)

Y comprobamos con el mapa de calor que el borrado se ha realizado correctamente, y qu eno nos dejamos ninguna variable con alta correlación.

In [ ]:
figure(figsize=(16,13))

sb.heatmap(data_train.corr(), annot=True)

De esta forma nos hemos quedado solo con 17 variables, teniendo así menos datos redundantes (lo cual es bastante malo para algunos algoritmos) ya que ahora cada una de las variables aporta bastante información propia.

### 3.2.2 Análisis univariado

Ahora, vamos a realizar un análisis univariado de las variables. Primero vamos a comprobar si nuestros datos tienen *outliers*. Para ello, nos hará falta dividir nuestro nuevo `data_train` en `X_train2` e `y_train2`, del mismo modo que hicimos anteriormente, y comprobar que están bien divididos.

In [ ]:
(X_train2, y_train2) = utils.divide_dataset(data_train, target="diagnosis")

X_train2.sample(5, random_state=seed)

In [ ]:
y_train2.sample(5, random_state=seed)

Una vez divididos, y comprobado que la división se ha realizado correctamente, vamos a representar un gráfico de cajas en el que podremos ver si contamos con outliers, además de la distribución de cada variable para la clase. Para ello, primero tenemos que estandarizar los datos, creando `data_est`.

Una versión usando *plotly* en lugar de *seaborn* sería:
> px.box(data_grafica, x="Variables", y="Estandarización", color="diagnosis")

Sin embargo, como en ocasiones *plotly* no funciona o relantiza mucho *Kaggle*, además de que en este caso se ven más claro los datos con *Seaborn*, nos quedaremos con esta última librería. 

In [ ]:
# Estandarizamos los datos
data_est = (X_train2 - X_train2.mean()) / (X_train2.std()) 

# Convertimos los datos para que puedan ser representados en la gráfica
data_grafica = pd.concat([y_train2, data_est], axis = 1)
data_grafica = pd.melt(data_grafica, id_vars = "diagnosis",
                     var_name = "Variables",
                     value_name = "Estandarización")

# Definimos el tamaño y el tipo de gráfica
pypl.figure(figsize = (25,10))
sb.boxplot(x="Variables", y="Estandarización", hue="diagnosis", data=data_grafica)

# Rotamos el nombre de las variables para que no se solapen
pypl.xticks(rotation = 90)

Podemos ver cómo contamos con *outliers* en todas las variables, por lo que los tendremos que eliminar posteriormente en el *pipeline*.

Además, en esta gráfica podemos ver cómo variables como `radius_mean` o `concavity_mean` pueden ser muy buenas para la clasificación ya que están muy diferenciadas entre `B` y `M`. Todo lo contrario pasa con otras variables como `texture_se` o `smoothness_se`, que será difícil que sean útiles para nuestro árbol de clasificación.

---

Ahora vamos a representar cada uno de los puntos de nuestro conjunto de test. En *plotly* podríamos hacer algo parecido usando:

> px.strip(data_grafica, x="Variables", y="Estandarización", color="diagnosis")

Sin embargo, también lo vamos a realizar con *Seaborn* ya que se ve mucho mejor.

In [ ]:
sb.catplot(x="Variables", y="Estandarización", hue="diagnosis", data=data_grafica, height=10, aspect=5/2, kind="swarm")
pypl.xticks(rotation = 90)

Podemos ver cómo efectivamente variables como `radius_mean`, `radius_se`, `radius_worst` o `concavity_mean` crean una división casi perfecta entre los casos Benignos y Malignos, mientras que otras como `texture_se`, `smoothness_se`, `fractal_dimension_mean` o `symmetry_se` están totalmente mezcladas.

El que haya variables que puedan crear una buena división por sí mismas nos dice que el modelo que generemos finalmente probablemente tenga un gran porcentaje de acierto, ya que simplemente usando esa clase se conseguiría un resultado decente.

---

Además, vamos a observar el histograma de las variables:

In [ ]:
utils.plot_histogram(data_train)

Podemos ver cómo todas las variables siguen más o menos una distribución normal. Sin embargo, algunas variables como `concavity_mean`, `radius_se`, `compactness_se` y `fractal_dimension_se` tienen una asimetría positiva bastante destacable. También podemos ver claramente los *outliers* en algunas variables como `concavity_se` o `radius_se`, entre otras.

## 4. Preprocesamiento de datos

---

### 4.1. Eliminar variables

Para eliminar las variables vamos a crear esta función simple (`EliminarVariables`) con `fit` y `transform` para poderlo usar en el *pipeline*. Simplemente tenemos que pasar por parámetro las variables que queremos borrar.

In [ ]:
class EliminarVariables():

    def __init__(self, columnas):
        self.columnas=columnas

    def fit(self, x, y=None):
        return self

    def transform(self, x, y=None):
        return x.drop(self.columnas, axis='columns', inplace=False)

### 4.2. Eliminación de *outliers*

Para eliminar los *outliers* vamos a usar una función (`outlier_rejection`) que cree un `IsolationForest` para detectarlos, y después pasarla por un `FunctionSampler` para poder incluirla en el *pipeline*.

In [ ]:
def outlier_rejection(X, y):

    model = IsolationForest(max_samples=100,
                            contamination=0.4,
                            random_state=seed)
    model.fit(X)
    y_pred = model.predict(X)
    return X[y_pred == 1], y[y_pred == 1]

elimOutliers = FunctionSampler(func=outlier_rejection)

### 4.3. Discretización

___

Vamos a utilizar un discretizador por `kmeans = 2`, ya que como hemos visto en el análisis de las variables, muchas de ellas como `radius_mean`, `radius_se`, `radius_worst`, `concavity_mean` o `concavity_worst` se pueden dividir casi perfectamente en dos partes, dejando a cada lado la clase mayoritaria. Además, usamos `kmeans` porque como los datos están desbalanceados, si partiésemos por ejemplo por la media seguramente esa partición sería peor. 

In [ ]:
discretizer = KBinsDiscretizer(n_bins=2, strategy="kmeans")

## 5. Aprendizaje y evaluación de modelos

---

### 5.1. Algoritmo Zero-R

Primero vamos a crear un clasificador `Zero-R`, que nos servirá como *baseline* para poder comparar con los valores que obtengamos de nuestros árboles de clasificación.

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

utils.evaluate(zero_r_model,
               X_train, X_test,
               y_train, y_test)

En el algoritmo *Zero-R* siempre se predecirá la clase mayoritaria, en este caso B (62,81% en los datos de entrenamiento). La tasa de acierto es por tanto un número muy cercano a dicho porcentaje (0,62573), ya que 0,6281 sería dicha tasa si utilizásemos el conjunto de datos de entrenamiento como test. Este rendimento es muy malo, ovbiamente, ya que tenemos que predecir si el cáncer de mama es benigno o maligno, y para todos los casos diríamos que es benigno.

---

### 5.2. Algoritmo *CART* (*Classification and Regression Trees*)

---

### 5.2.1. Algoritmo *CART* sin eliminar variables y *outliers*.

Ahora vamos a probar los modelos basados en árboles de clasificación sin discretizar.

Primero creamos nuestro clasificador `DecisionTreeClassifier`, que usaremos para todas las demás modelos de árboles de clasificación. Como hiperparámetros incluiremos la semilla, que garanzita que los resultados sean reproducibles; vamos a establecer como criterio la entropía en lugar de Gini, ya que es el que más hemos usado en asignaturas anteriores y en la parte de teoría de Minería de Datos; y establecemos a 4 el mínimo de hojas para no realizar un sobreajuste demasiado grande en las hojas del árbol.

Ahora, vamos a comprobar el rendimiento del clasificador usando la base de datos original:

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed,
                                    criterion='entropy',
                                    min_samples_leaf = 5)

utils.evaluate(tree_model,
               X_train, X_test,
               y_train, y_test)

Podemos ver cómo obtenemos un *accuracy* del 93,567%, lo cual es una mejora muy grande con respecto a `Zero-R`. Tiene sentido, ya que como hemos visto en el análisis exploratorio de los datos había varias variables que podían dividir bastante bien los datos dependiendo de la clase `B`o `M` obteniendo poco error.

---

También podemos representar el árbol que se ha generado usando la librería `graphviz`, que es el siguiente:

In [ ]:
dot_data = tree.export_graphviz(tree_model, out_file=None, 
                         feature_names=list(X_train),  
                         class_names=["B", "M"],  
                         filled=True, rounded=True,  
                         special_characters=True)

graphviz.Source(dot_data)

Podemos ver cómo no es excesivamente grande, y utiliza varias variables como `permieter_worst` o `concave points_worst` que nosotros hemos eliminado en el análisis exploratorio de datos, por lo que si no hemos realizado bien el proceso de selección de variables, el resultado al borrarlas podría verse muy afectado.

---

### 5.2.2. Algoritmo CART eliminando variables y outliers.

Ahora vamos a realizar un *pipeline* realizando la eliminación de las variables que tenían una gran correlación en el análisis exploratorio, y de los outliers que también encontramos en dicho análisis antes de evaluar el árbol de clasificación.

In [ ]:
sinOutliers_tree_model = make_pipeline(EliminarVariables(borrado),
                                       elimOutliers,
                                       tree_model)

utils.evaluate(sinOutliers_tree_model,
               X_train, X_test,
               y_train, y_test)

Vemos como la tasa de aciertos ha disminuido del 93,567% al 90,058%. No es una diferencia muy grande teniendo en cuenta que los conjuntos de entrenamiento y test no son muy grandes, por lo que el resultado se puede ver bastante afectado dependiendo de la semilla que hayamos definido. Aún así, dicho resultado tiene sentido ya que los árboles de clasificación con variables numéricas no se ven afectados por el hecho de tener variables muy correlacionadas entre sí, ni les castiga excesivamente la presencia de algunos outliers.

Así, con poco que eliminemos alguna variable que sea un algo mejor clasificando que las que dejamos, o que por lo que sea viene mejor para los datos de test que tenemos, ya reduciremos sensiblemente la tasa de acierto.

---

Al igual que antes, también vamos a representar el árbol:

In [ ]:
dot_data = tree.export_graphviz(tree_model, out_file=None, 
                         feature_names=list(X_train2),  
                         class_names=["B", "M"],  
                         filled=True, rounded=True,  
                         special_characters=True)

graphviz.Source(dot_data)

Como podemos ver, solo aparecen las variables que dejamos en el análisis exploratorio, por lo que el *pipeline* parece haber funcionado correctamente. Este árbol es prácticamente del mismo tamaño que el anterior, ya que como en nuestra base de datos tenemos pocas instancias, el pasar de 30 a 17 variables predictoras no hace que disminuya el tamaño de éste ya que se queda antes sin instancias que sin variables. Con una base de datos más grande, seguramente sí que podríamos conseguir un modelo más compacto que sobreajuste menos.

___

### 5.3. Algoritmo CART (Classification and Regression Trees) con discretización

---

### 5.3.1. Algoritmo CART discretizando, pero sin eliminar variables y outliers.

Ahora vamos a comprobar el rendimiento del árbol de decisión cuando le aplicamos el discretizador por `kmeans` con 2 *bins*. 

In [ ]:
discretize_tree_model = make_pipeline(discretizer, 
                                      tree_model)

utils.evaluate(discretize_tree_model,
               X_train, X_test,
               y_train, y_test)

Podemos ver cómo hemos obtenido un porcentaje de acierto del 92,398%, algo menor que el 93,567% que obteníamos sin discretizar. Con esta variación tan pequeña no podemos obtener ninguna conclusión sobre cuál es mejor, aunque es un buen valor teniendo en cuenta que los árboles de clasificación trabajan especialmente bien con variables continuas.

___

### 5.3.2. Algoritmo *CART* discretizando y eliminando variables y *outliers*.

Por último, vamos a aplicar la eliminación de variables y *outliers* junto con la discretización.

In [ ]:
discretize_tree_model = make_pipeline(EliminarVariables(borrado),
                                      elimOutliers, 
                                      discretizer, 
                                      tree_model)

utils.evaluate(discretize_tree_model,
               X_train, X_test,
               y_train, y_test)

Podemos ver que en este caso el porcentaje de acierto ha bajado de un 92,398% a un 91,913%. Este resultado es más extraño que los anteriores ya que para la discretización sí que es especialmente útil la eliminación de outliers que no alteren artificialmente los intervalos, por lo que podemos presuponer que esta pequeña reducción del *accuracy* viene dada simplemente por tener mala suerte con estos datos en concreto o con la semilla escogida en los casos que se use la aleatorización.